# IQ Constellation Data Explore
This Notebook explores the following approach
1. Apply KMeans Clustering to map blobs to clusters
2. Extract Blob Shape by applying Covariance Error Ellipse
3. Plot new metrics next to raw blob plot
4. Use statistical methods to determine Interference/Phase Noise/Amplifier Gain Noise
5. Write report to S3

A next step to this solution could be applying a Decision Tree Classifier to make an inference based on the shape metrics extracted from Covariance Error Ellipse

conda_python3, ml.m5.xlarge

In [20]:
import warnings
warnings.filterwarnings("ignore")

import sagemaker
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import boto3
import time
import os
from sklearn.cluster import KMeans
from matplotlib.patches import Ellipse
from matplotlib.path import Path
from time import gmtime, strftime
import io

from utility import confidence_ellipse, line_distance

In [21]:
# Constants
STD_DEV = 2.5

## Extract Blob Shape Metrics, Covariance Error Ellipse

In [22]:
def process(timestamps, data_type, flag=False):
    result_buffer = []
    for i in range(len(timestamps)):

        X = timestamps[i]
        real = np.real(X)
        imag = np.imag(X)
        X = np.concatenate([real.reshape(-1, 1), imag.reshape(-1, 1)], axis=1)
        
        # Map Blobs into clusters
        y_pred = KMeans(n_clusters=CLUSTERS).fit_predict(X)

        # Plot with clusters
        # axs[i][0].scatter(X[:, 0], X[:, 1], c=y_pred)
        
        # Build object of {cluster, [x,y]}
        Y = []
        for j in range(CLUSTERS):
            Y.append([])
        for j in range(len(X)):
            Y[y_pred[j]].append(X[j])
        if (flag):
            Y = np.array(np.array([np.array(Y) for Y in Y], dtype=object), dtype=np.float32)
        else:
            Y = np.array(np.array([np.array(Y) for Y in Y], dtype=object), dtype=object)

        density = []
        ratio = []
        major_axis = []
        minor_axis = []
        major_rotation = 0

        for j in range(len(Y)):
            # Generate Ellipse
            ellipse = confidence_ellipse(Y[j][:,0], Y[j][:,1], n_std=STD_DEV)
            maj_axis = [ellipse['center'][0], ellipse['center'][0]+ellipse['major_axis'][0]], [ellipse['center'][1], ellipse['center'][1]+ellipse['major_axis'][1]]
            min_axis = [ellipse['center'][0], ellipse['center'][0]+ellipse['minor_axis'][0]], [ellipse['center'][1], ellipse['center'][1]+ellipse['minor_axis'][1]]
            res = line_distance(maj_axis, min_axis, (0,0))
            if (not res):
                major_rotation += 1
            major_axis.append(maj_axis)
            minor_axis.append(min_axis)
            density.append(ellipse['density'])
            ratio.append(ellipse['ratio'])

            # Add to plot
            # axs[i][1].add_patch(ellipse['ellipse'])
            # axs[i][1].plot([ellipse['center'][0], ellipse['center'][0]+ellipse['major_axis'][0]], [ellipse['center'][1], ellipse['center'][1]+ellipse['major_axis'][1]], color='r', linewidth=2)
            # axs[i][1].plot([ellipse['center'][0], ellipse['center'][0]+ellipse['minor_axis'][0]], [ellipse['center'][1], ellipse['center'][1]+ellipse['minor_axis'][1]], color='g', linewidth=2)        
            # axs[i][1].plot(Y[j][:,0], Y[j][:,1], 'o')
            
        # Analyze features
        major_axis_np = np.array(major_axis)
        minor_axis_np = np.array(minor_axis)
        density_np = np.array(density)
        ratio_np = np.array(ratio)
        result_buffer.append( {
            'density_avg': np.mean(density_np, axis=0), 
            'density_std': np.std(density_np, axis=0), 
            'ratio_avg': np.mean(ratio_np, axis=0), 
            'ratio_std': np.std(ratio_np, axis=0),
            # 'major_axis_avg': np.mean(major_axis_np, axis=0), 
            # 'major_axis_std': np.std(major_axis_np, axis=0),
            # 'minor_axis_avg': np.mean(minor_axis_np, axis=0), 
            # 'minor_axis_std': np.std(minor_axis_np, axis=0),            
            'rotation': (major_rotation /CLUSTERS), 
            'blobs': CLUSTERS, 
            'class': data_type})
    return result_buffer

# modcod = 'qpsk'
# CLUSTERS = 4


# timestamps = []
# data_type = 'compression'
# file_path = modcod + '/' + data_type
# fn = os.listdir('./synth-data/%s' % file_path)[:2]
# for f in fn:
#     # print(np.load('./synth-data/%s/%s' % ( file_path,f),allow_pickle=True))
#     if (data_type == 'compression'):
#         timestamps.append(np.load('./synth-data/%s/%s' % ( file_path,f), allow_pickle=True))
#     else:
#         timestamps.append(np.fromfile(open('./synth-data/%s/%s' % ( file_path,f)), dtype=np.complex64))
# fig, axs = plt.subplots(nrows=len(timestamps), ncols=2, figsize=(20, 20))
# result = process(timestamps, data_type, True)
# res_df = pd.DataFrame.from_dict(result)



# plt.show()

In [23]:
def configuration(data_type, clusters, modcod):
    timestamps = []
    file_path = modcod + '/' + data_type
    fn = os.listdir('../data_generation/generator/data/%s' % file_path)
    for f in fn:
        timestamps.append(np.fromfile(open('../data_generation/generator/data/%s/%s' % ( file_path,f)), dtype=np.complex64))
    result = process(timestamps, data_type, False)
    return pd.DataFrame.from_dict(result)

In [24]:
# Opening JSON file
# ------------------------------------ qpsk ------------------------------------
modcod = 'qpsk'
CLUSTERS = 4

timestamps = []
data_type = 'compression'
file_path = modcod + '/' + data_type
fn = os.listdir('../data_generation/generator/data/%s' % file_path)
for f in fn:
     timestamps.append(np.load('../data_generation/generator/data/%s/%s' % ( file_path,f), allow_pickle=True))
# fig, axs = plt.subplots(nrows=len(timestamps), ncols=2, figsize=(20, 20))
result = process(timestamps, data_type, True)
res_df_0 = pd.DataFrame.from_dict(result)

res_df_1 = configuration(data_type='normal', clusters=CLUSTERS, modcod=modcod)
res_df_2 = configuration(data_type='phase_noise', clusters=CLUSTERS, modcod=modcod)
res_df_3 = configuration(data_type='interference', clusters=CLUSTERS, modcod=modcod)

# ------------------------------------ 8psk ------------------------------------

modcod = '8psk'
CLUSTERS = 8

res_df_4 = configuration(data_type='normal', clusters=CLUSTERS, modcod=modcod)
res_df_5 = configuration(data_type='phase_noise', clusters=CLUSTERS, modcod=modcod)
res_df_6 = configuration(data_type='interference', clusters=CLUSTERS, modcod=modcod)

# ------------------------------------ 16apsk ------------------------------------

modcod = '16apsk'
CLUSTERS = 16

res_df_7 = configuration(data_type='normal', clusters=CLUSTERS, modcod=modcod)
res_df_8 = configuration(data_type='phase_noise', clusters=CLUSTERS, modcod=modcod)
res_df_9 = configuration(data_type='interference', clusters=CLUSTERS, modcod=modcod)

# ------------------------------------ 32apsk ------------------------------------

modcod = '32apsk'
CLUSTERS = 32

res_df_10 = configuration(data_type='normal', clusters=CLUSTERS, modcod=modcod)
res_df_11 = configuration(data_type='phase_noise', clusters=CLUSTERS, modcod=modcod)
res_df_12 = configuration(data_type='interference', clusters=CLUSTERS, modcod=modcod)

results = pd.concat([res_df_0, res_df_1, res_df_2, res_df_3, res_df_4, res_df_5, res_df_6, res_df_7, res_df_8,res_df_9, res_df_10, res_df_11,res_df_12])
blobs_encoded = pd.get_dummies(results['blobs'], prefix='blobs')
results = pd.concat([results, blobs_encoded], axis=1)
results = results.drop('blobs', axis=1)

In [25]:
results.head()

,density_avg,density_std,ratio_avg,ratio_std,rotation,class,blobs_4,blobs_8,blobs_16,blobs_32
0,43547.288970,37801.835055,2.894907,1.073702,0.25,compression,True,False,False,False
1,12657.028852,2719.201631,3.935051,0.231238,0.50,compression,True,False,False,False
2,27313.478924,5557.233032,3.213635,0.395462,0.25,compression,True,False,False,False
3,13533.765908,2819.171889,3.879718,0.142895,0.50,compression,True,False,False,False
4,13999.897139,2572.639980,3.993220,0.170502,1.00,compression,True,False,False,False


In [26]:
results.to_csv('training_data.csv')